In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [21]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_out = pd.read_csv("test.csv")
test['SalePrice'] = 0

In [3]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,0
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,0
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,0
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,0
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,0


In [4]:
def correlation(ds):
    numeric_features = train.select_dtypes(include=[np.number]) #catch numeric columns
    numeric_features.corr()['SalePrice'].sort_values(ascending = False)[:10]

In [5]:
def data_clean(ds,test):
    #print(ds.head())
    ds['MSZoning'].replace({'C (all)': 0, 'RM':1, 'RH':2, 'RL':3, 'FV':4}, inplace = True) 
    numeric_features = ds.select_dtypes(include=[np.number])
    corr = numeric_features.corr()
    #print (corr['SalePrice'].sort_values(ascending=False))
    a = numeric_features.corr()['SalePrice'].sort_values(ascending=False)[:10] #pega as 10 maiores correlações
    print(a)
    index = a.index #retira o index delas 
    ds[index] = ds[index].fillna(0)
    ds = ds[index]
    ds = ds.dropna()
    
    test['MSZoning'].replace({'C (all)': 0, 'RM':1, 'RH':2, 'RL':3, 'FV':4}, inplace = True)
    test[index] = test[index].fillna(0)
    test = test[index]
    test = test.dropna()
    #print(ds.isnull().sum())
    
    return ds,test

In [69]:
#Not numeric start
test['SalePrice']

KeyError: 'SalePrice'

In [45]:
numeric_features = test.select_dtypes(include=[np.number])
numeric_features = numeric_features.columns
ds_not_num = pd.DataFrame()
test.drop(numeric_features,axis='columns', inplace=True)

In [87]:
for i in test.columns:
    a = train[[i, 'SalePrice']].groupby(i).mean()
    #print(a['SalePrice'].sort_values(ascending=True))
    print(train[[i]].groupby(i).mean())
    index = i.index
    for z in index:
        print(z)

Empty DataFrame
Columns: []
Index: [C (all), FV, RH, RL, RM]


ValueError: substring not found

In [6]:
###Teste
(train,test) = data_clean(train,test)
#test = data_clean(test)

SalePrice       1.000000
OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
FullBath        0.560664
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
Name: SalePrice, dtype: float64


In [31]:
train.groupby('GarageCars').mean()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
GarageCars,,,,,,,,,,,,,,,,,,,,,
0,707.012346,71.790123,60.405405,7469.679012,4.604938,5.530864,1942.481481,1975.172840,13.777778,240.382716,...,39.555556,36.172840,37.530864,0.000000,0.000000,0.000000,53.703704,6.111111,2007.876543,103317.283951
1,725.200542,50.230352,62.548077,8636.685637,5.189702,5.964770,1948.243902,1971.257453,39.753388,333.230352,...,65.479675,23.449864,38.750678,0.867209,13.078591,1.560976,81.165312,6.265583,2007.850949,128116.688347
2,734.627427,60.934466,70.624611,11076.458738,6.251214,5.510922,1978.787621,1988.411408,100.756426,468.156553,...,98.776699,51.533981,14.899272,5.283981,17.461165,3.515777,34.750000,6.291262,2007.836165,183851.663835
3,725.292818,44.475138,85.798817,13063.127072,7.950276,5.104972,1997.287293,2000.850829,287.394444,643.933702,...,155.906077,77.773481,11.331492,1.679558,15.331492,3.066298,3.093923,6.662983,2007.618785,309636.121547
4,1010.400000,92.000000,76.000000,14232.000000,5.400000,5.200000,1955.600000,1983.200000,143.400000,593.600000,...,124.000000,0.000000,77.200000,0.000000,0.000000,0.000000,0.000000,6.600000,2008.000000,192655.800000


In [8]:
train.isnull().sum()

SalePrice       0
OverallQual     0
GrLivArea       0
GarageCars      0
GarageArea      0
TotalBsmtSF     0
1stFlrSF        0
FullBath        0
TotRmsAbvGrd    0
YearBuilt       0
dtype: int64

In [9]:
X = train.drop(['SalePrice'], axis='columns')
y = train['SalePrice']

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [11]:
#Random Forest Classifier
model2=RandomForestClassifier(n_estimators =100)
model2.fit(X_train,y_train)
pred_2=model2.predict(X_test)
rcf=accuracy_score(pred_2,y_test);
print(rcf)   

0.0136986301369863


In [12]:
#Logistic Regression
model1 = LogisticRegression(solver='lbfgs', max_iter=1000)
model1.fit(X_train, y_train)
pred = model1.predict(X_test)
log_acc = accuracy_score(pred, y_test)
print(log_acc)

0.0136986301369863


C:\Users\Danilo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
#index = index.drop('SalePrice')

In [14]:
test.drop('SalePrice',axis = 'columns', inplace = True)

In [15]:
model2.fit(X,y)
pred = model2.predict(test)

In [17]:
test.isnull().sum()

OverallQual     0
GrLivArea       0
GarageCars      0
GarageArea      0
TotalBsmtSF     0
1stFlrSF        0
FullBath        0
TotRmsAbvGrd    0
YearBuilt       0
dtype: int64

In [18]:
result = pd.DataFrame()
result['Id'] = test_out.Id
result['SalePrice'] = pred

In [19]:
result.head(5)

,Id,SalePrice
0,1461,109500
1,1462,155000
2,1463,181000
3,1464,181000
4,1465,180000


In [ ]:
result.to_csv('sub.csv', index = False)